First, we need to import the libraries needed to have access to functions.py and various other libraries

In [1]:
import sys
import os

We append our to working path the ../src path, where functions.py lies

In [2]:

# need to be able to import src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# absolute path to the src directory
src_path = os.path.abspath("../src")

# Add src directory to system path
sys.path.append(src_path)

# get the /MLCBAss1/Assignment-2 path
ass_path=sys.path[-2]

print(ass_path)

/home/sdi1700187/MLCBAss2/Assignment-2


We import everything from functions.py. Also the first two line ensure that everytime we change something to functions.py, this change can be "seen" here

In [3]:
%load_ext autoreload
%autoreload 2

from functions import *